In [1]:
# 必要なライブラリのインポート
import sys
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
# config 読み込み
base_dir = Path.cwd().parent.parent
config_path = base_dir / "config"
sys.path.append(str(config_path))

from config import (
    RANDOM_SEED,
    CLASS_PRIORS,
    FEATURE_RANGES,
    NOISE
)

rng = np.random.default_rng(RANDOM_SEED)

In [3]:
# パスを取得し、cdvデータの読込み込み
csv_dir = base_dir / "assets" / "csv_data"
input_path  = csv_dir / "主要被害公園エリア_座標結合済み.csv"
output_path = csv_dir / "主要被害公園エリア_leaftemp付与.csv"
df = pd.read_csv(input_path)

In [4]:
# クラスを priors からサンプリング
classes = list(CLASS_PRIORS.keys())
priors  = np.array(list(CLASS_PRIORS.values()), dtype=float)
priors  = priors / priors.sum() 
labels  = rng.choice(classes, size=len(df), p=priors)

# leaf_temp_mean をクラスごとの範囲から一様乱数で生成 → ノイズ → クリップ 
vals = np.empty(len(df), dtype=float)

for c in classes:
    m = (labels == c)
    if m.any():
        lo, hi = FEATURE_RANGES[c]["leaf_temp_mean"]
        vals[m] = rng.uniform(lo, hi, size=m.sum())

# ガウスノイズ（平均0, σ=config.NOISE.get("leaf_temp_mean", 0.0)）
sigma = float(NOISE.get("leaf_temp_mean", 0.0) or 0.0)
if sigma > 0:
    vals += rng.normal(0.0, sigma, size=len(vals))

spill_p = float(NOISE.get("spillover_rate", 0.0) or 0.0)
if spill_p > 0:
    k = int(spill_p * len(vals))
    if k > 0:
        idx = rng.choice(len(vals), size=k, replace=False)
        sigma_sp = float(NOISE.get("leaf_temp_mean_spill", 0.0) or 0.0)
        if sigma_sp > 0:
            vals[idx] += rng.normal(0.0, sigma_sp, size=k)

# 物理レンジでクリップ（従来実装に合わせて）
vals = np.clip(vals, 15.0, 50.0)

# 結果をデータフレームに追加
df["leaf_temp_mean"] = vals
out = df[["park_name", "経度", "緯度", "leaf_temp_mean"]].copy()
out.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"CSVを出力完了: {output_path}")
display(out.head())

CSVを出力完了: c:\Users\kyous\OneDrive\デスクトップ\ハッカソン\tokyo-tree-doctor_program\tokyo-tree-doctor\ml\assets\csv_data\主要被害公園エリア_leaftemp付与.csv


,park_name,経度,緯度,leaf_temp_mean
0,桜ヶ丘公園,139.456775,35.639011,29.828754
1,桜ヶ丘公園,139.456777,35.638741,24.667088
2,桜ヶ丘公園,139.457100,35.640364,31.532105
3,桜ヶ丘公園,139.457101,35.640094,28.982761
4,桜ヶ丘公園,139.457103,35.639823,26.972904
